### Objetivo
Analizar la base de datos de una aplicación de lectura para responder las siguientes preguntas:

¿Cuántos libros fueron publicados después del 1 de enero de 2000?

¿Cuál es el número de reseñas y la calificación promedio por libro?

¿Qué editorial ha publicado más libros con más de 50 páginas?

¿Qué autor tiene la calificación promedio más alta (considerando libros con al menos 50 calificaciones)?

¿Cuál es el número promedio de reseñas de texto entre usuarios que calificaron más de 50 libros?

In [1]:
# Importar librerías
import pandas as pd
from sqlalchemy import create_engine

# Configuración de conexión
db_config = {
    'user': 'practicum_student',  # usuario
    'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7',  # contraseña
    'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
    'port': 5432,  # puerto
    'db': 'data-analyst-final-project-db'  # base de datos
}

# Crear la cadena de conexión
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)

# Crear el motor
engine = create_engine(connection_string, connect_args={'sslmode': 'require'})

# Probar conexión — ver tablas disponibles
query = "SELECT table_name FROM information_schema.tables WHERE table_schema='public';"
tables = pd.read_sql(query, con=engine)
display(tables)

,table_name
0,ratings
1,advertisment_costs
2,authors
3,orders
4,reviews
5,visits
6,books
7,users
8,publishers


In [7]:
#Libros publicados después del 1 de enero de 2000
query = """
SELECT COUNT(*) AS books_after_2000
FROM books
WHERE publication_date > '2000-01-01';
"""
books_after_2000 = pd.read_sql(query, con=engine)
display(books_after_2000)

#Permite saber cuántos libros modernos (posteriores al 2000) están disponibles en el catálogo.

,books_after_2000
0,819


In [8]:
#Número de reseñas y calificación promedio por libro

query = """
SELECT 
    b.book_id,
    b.title,
    COUNT(DISTINCT r.review_id) AS num_reviews,
    ROUND(AVG(rt.rating), 2) AS avg_rating
FROM books b
LEFT JOIN reviews r ON b.book_id = r.book_id
LEFT JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY b.book_id, b.title
ORDER BY avg_rating DESC;
"""
reviews_and_ratings = pd.read_sql(query, con=engine)
display(reviews_and_ratings.head())

#Esta tabla muestra qué libros son los más calificados y cuáles tienen más participación de usuarios.

,book_id,title,num_reviews,avg_rating
0,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.0
1,901,The Walking Dead Book One (The Walking Dead #...,2,5.0
2,390,Light in August,2,5.0
3,972,Wherever You Go There You Are: Mindfulness Me...,2,5.0
4,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.0


In [9]:
#Editorial con más libros de más de 50 páginas
query = """
SELECT 
    p.publisher,
    COUNT(b.book_id) AS total_books
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY total_books DESC
LIMIT 1;
"""
top_publisher = pd.read_sql(query, con=engine)
display(top_publisher)

#Nos dice qué editorial produce más libros extensos (más de 50 páginas), excluyendo folletos o publicaciones pequeñas.

,publisher,total_books
0,Penguin Books,42


In [10]:
#Autor con la calificación promedio más alta (mínimo 50 calificaciones)
query = """
SELECT 
    a.author,
    ROUND(AVG(rt.rating), 2) AS avg_rating,
    COUNT(rt.rating_id) AS num_ratings
FROM authors a
JOIN books b ON a.author_id = b.author_id
JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY a.author
HAVING COUNT(rt.rating_id) >= 50
ORDER BY avg_rating DESC
LIMIT 1;
"""
top_author = pd.read_sql(query, con=engine)
display(top_author)

#Ayuda a identificar al autor más popular y mejor valorado entre los lectores frecuentes.

,author,avg_rating,num_ratings
0,Diana Gabaldon,4.3,50


In [11]:
#Promedio de reseñas textuales entre usuarios que calificaron más de 50 libros
query = """
WITH active_users AS (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) > 50
)
SELECT 
    ROUND(AVG(user_review_count), 2) AS avg_reviews_per_user
FROM (
    SELECT r.username, COUNT(r.review_id) AS user_review_count
    FROM reviews r
    JOIN active_users u ON r.username = u.username
    GROUP BY r.username
) sub;
"""
avg_reviews_active_users = pd.read_sql(query, con=engine)
display(avg_reviews_active_users)

#Muestra cuántas reseñas textuales dejan en promedio los usuarios más activos (los que califican muchos libros).

,avg_reviews_per_user
0,24.33
